In [1]:
#################################
# filenames
#################################

number = 22

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            #time.sleep(1.1)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(30)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(30)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    time.sleep(1.1)
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                    # this is not where this block belongs
                                    # nesting it under "else" above made failures generate "[]" and 
                                    #    only appended new songs to song_list, ignoring those already in url_dict
#                                     #driver.close()
#                                     # clean up and append song to list
#                                     my_json[i].pop('section', None)
#                                     my_json[i]['url'] = url
#                                     #f.write('\n'+str(my_json[i])+',')
#                                     song_list.append(my_json[i])

                                # this is where the above block really belongs 
                                #     so song_list gets every song, even if the url is in url_dict
                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1.1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(30)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(30)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1.1)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(30)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(30)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1.1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1.1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo[:100])
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615158932.610907
500
1/500 1,6,2,57/3
histo1 1
2/500 265,365,265,365
histo1 1
3/500 542,47,5,164
histo1 1
4/500 5,1,b7,67
histo1 2
5/500 7,6,7,4
histo1 2
6/500 265,365,265,5/5
histo1 1
7/500 264,5,543,1
histo1 1
8/500 5/5,5/2,4,1
histo1 1
9/500 5/6,66,364,6
histo1 1
10/500 b7,4,b6,b3
histo1 2
11/500 5,4,2,5
histo1 17
12/500 6,565/6,543/4,4
histo1 1
13/500 642,57/4,4,57/6
histo1 1
14/500 27,16,4,142
histo1 1
15/500 4,16,b3,b76
histo1 1
16/500 1,542/4,46,C465
histo1 1
17/500 66,6,642,6
histo1 1
18/500 1,164,5,164
histo1 1
19/500 4,5,67,b77
histo1 1
20/500 5/6,5/5,6,5
histo1 1
21/500 1,17,M17,1
histo1 1
22/500 4,5,b7,6
histo1 3
23/500 57,1,b464,1
histo1 1
24/500 164,464,1,5
histo1 1
25/500 5,57/6,56/6,6
histo1 1
26/500 56,5,464,4
histo1 1
27/500 57/6,66,6,5
histo1 1
28/500 1,564,6,1
histo1 1
29/500 37,67,47,57/6
histo1 1
30/500 2,1,5,542
histo1 1
31/500 3,4,442,2
histo1 2
32/500 57/4,464,1,57/4
histo1 1
33/500 4,5/5,M17,5
histo1 1
34/500 57/5,1,5,4
histo1 1
35/500 57/2,4,7,3
histo1 1
36/

histo1 2
291/500 4,164,2,264
histo1 1
292/500 1,4,5,M4
histo1 1
293/500 5/5,564,56/3,16
histo1 1
294/500 5,1,b7,6
histo1 14
295/500 464,364,6,564
histo1 1
296/500 5,4,3,464
histo1 2
297/500 37,57/2,47,37
histo1 1
298/500 6,7/6,6,4/7
histo1 1
299/500 365,17,67,665
histo1 1
300/500 6,b6,b4,3
histo1 1
301/500 264,5,1,56
histo1 2
302/500 6,4,27,3
histo1 1
303/500 5,3,5/6,4
histo1 3
304/500 2,6,664,4
histo1 4
305/500 464,564,664,264
histo1 1
306/500 365,57,1,242
histo1 1
307/500 46,564,1,46
histo1 1
308/500 46,b46,164,5/5
histo1 1
309/500 26,77,57/6,4
histo1 1
310/500 6,26,L26,1
histo1 1
311/500 57,7,1,564
histo1 1
312/500 b7,1,4,1
histo1 5
313/500 1,164,4,3
histo1 1
314/500 b7,46,56,1
histo1 1
315/500 56/6,36,L26,4
histo1 1
316/500 27,77,Y7,Y27
histo1 1
317/500 564,164,264,6
histo1 1
318/500 6,37,C37,27
histo1 1
319/500 16,67,2,16
histo1 1
320/500 6,4,265,4
histo1 1
321/500 47,5/6,6,36
histo1 2
322/500 1,57/4,4,b4
histo1 6
323/500 16,5,76/5,5
histo1 1
324/500 642,27,243,37
histo1 1
325/500

In [ ]:
##########################
# list errors here
##########################
182/500 27,5,3,6
2. Unable to extract for 27 5 3 6 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [3]:
##########################
# test errors here
##########################
mykey = '27,5,3,6'
page = 1
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [2]:
# ##########################
# # fix histo here
# ##########################
# histo[20] -= 1
# print(histo[:100])
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 363, 66, 26, 10, 3, 4, 4, 2, 2, 3, 1, 0, 1, 2, 1, 2, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
